In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

def load_process_data(file_path, id_listed_shares):
    # Carregar os dados
    df = pd.read_csv(file_path, sep=',')

    # Renomeie as colunas
    df.rename(columns={'Data': 'date', 'Último': 'last_value', 'Abertura': 'opening','Máxima': 'high', 'Mínima': 'low', 'Vol.': 'trading_volume','Var%': 'percentage_change'}, inplace=True)

    # Adicione a coluna "id_listed_shares"
    df['id_listed_shares'] = id_listed_shares

    return df

# Caminhos dos arquivos e IDs correspondentes
path = '/content/drive/My Drive/Colab Notebooks/PI-6-SEMESTRE'

files_and_ids = [(path + '/EMBR3_Dados_Historicos.csv', 1),(path + '/TOTS3_Dados_Historicos.csv', 2),(path + '/BBAS3_Dados_Historicos.csv', 3),(path + '/VIVA3_Dados_Historicos.csv', 4),(path + '/KLBN3_Dados_Historicos.csv', 5),(path + '/PETR4_Dados_Historicos.csv', 6)]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import re

# Listas para armazenar os DataFrames
dfs = []

# Carregar e processar os dados para cada arquivo
for file_path, id_listed_shares in files_and_ids:
    df = load_process_data(file_path, id_listed_shares)
    dfs.append(df)

# Concatenar os DataFrames em um único DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Exibir o DataFrame resultante
print('\n\nHistórico Consolidado: --------------------')

# #Verificando valor nulo
merged_df['trading_volume'] = merged_df['trading_volume'].fillna('0,0')
merged_df.isnull().sum()


def definir_perfil_investimento(row):
    percentage_change = row['percentage_change']

    # Convertendo o trading_volume para número
    trading_volume = row['trading_volume']
    if 'M' in trading_volume:
        trading_volume = float(re.sub(r'[^\d.]', '', trading_volume)) * 1000000  # Convertendo milhões para número
    elif 'K' in trading_volume:
        trading_volume = float(re.sub(r'[^\d.]', '', trading_volume)) * 1000  # Convertendo milhares para número
    else:
        trading_volume = float(trading_volume.replace(',', '.'))

    merged_df.at[row.name, 'trading_volume'] = trading_volume

    # Calculando volatilidade
    volatilidade = row['high'] - row['low']

    # Verificando a liquidez
    if trading_volume > 1000000:  # Volumes acima de 1 milhão são considerados alta liquidez
        liquidez = "alta"
    else:
        liquidez = "baixa"

    # Verificando a tendência de preço
    if percentage_change >= 0:
        tendencia_preco = "crescente"
    else:
        tendencia_preco = "decrescente"

    # Verificando o retorno potencial
    if abs(percentage_change) >= 0.8:  # Retornos absolutos acima de 0.5 são considerados "altos"
        retorno_potencial = "alto"
    else:
        retorno_potencial = "baixo"

    # Definindo o perfil de investimento com base nos atributos
    if liquidez == "alta" and tendencia_preco == "crescente":
        if retorno_potencial == "baixo":
            return 1 #"Perfil Conservador"
        elif retorno_potencial == "alto":
            return 3 #"Perfil Agressivo"
    elif liquidez == "baixa" and tendencia_preco == "crescente" and retorno_potencial == "alto":
        return 3 #"Perfil Agressivo"

    # Caso nenhuma das condições anteriores seja atendida, atribuímos o perfil como Moderado
    return 2 #"Perfil Moderado"


# Lista das colunas que precisam ser convertidas para valores numéricos
cols_to_convert = ['last_value', 'opening', 'high', 'low', 'percentage_change']
# Convertendo as colunas para valores numéricos, substituindo vírgulas por pontos e removendo o sinal de porcentagem
merged_df[cols_to_convert] = merged_df[cols_to_convert].apply(lambda x: pd.to_numeric(x.str.replace(',', '.').str.rstrip('%')))

# Formata o campo "date" para YYY-MM-DD
merged_df['date'] = pd.to_datetime(merged_df['date'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')

# Adicionar a coluna de perfil de investimento
merged_df['id_profile'] = merged_df.apply(definir_perfil_investimento, axis=1)
display(merged_df)

# Exportar o DataFrame para um arquivo CSV no Google Drive
merged_df.to_csv('/content/drive/My Drive/Colab Notebooks/PI-6-SEMESTRE/listed_share_history.csv', index=False)
print("Exportação concluída. Verifique se o arquivo CSV foi salvo corretamente no Google Drive.")





Histórico Consolidado: --------------------


,date,last_value,opening,high,low,trading_volume,percentage_change,id_listed_shares,id_profile
0,2024-04-12,32.45,32.40,32.50,32.22,88660000.0,-0.52,1,2
1,2024-04-11,32.62,32.71,32.80,31.95,345000000.0,-0.67,1,2
2,2024-04-10,32.84,32.35,32.96,32.30,525000000.0,0.89,1,3
3,2024-04-09,32.55,32.51,32.68,31.95,358000000.0,0.18,1,1
4,2024-04-08,32.49,32.30,32.62,31.86,427000000.0,0.34,1,1
...,...,...,...,...,...,...,...,...,...
6408,2020-01-08,30.50,30.69,30.77,30.24,4822000000.0,-0.62,6,2
6409,2020-01-07,30.69,30.82,30.88,30.47,3282000000.0,-0.39,6,2
6410,2020-01-06,30.81,30.43,30.94,29.95,8184000000.0,1.18,6,3
6411,2020-01-03,30.45,30.88,31.24,30.45,7160000000.0,-0.81,6,2


Exportação concluída. Verifique se o arquivo CSV foi salvo corretamente no Google Drive.


In [ ]:
#Confirmando valores da classe
# 1 = "Perfil Conservador" / 2 = "Perfil Moderado" / 3 = "Perfil Agressivo"
print(merged_df['id_profile'].value_counts())

id_profile
2    3091
3    2251
1    1071
Name: count, dtype: int64


In [ ]:
# Separar os dados em features (X) e variável alvo (y)
X = merged_df.drop('id_profile', axis=1).drop('date', axis=1)
y = merged_df['id_profile']

# Métricas
seed = 7
scoring = 'accuracy'

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test =  model_selection.train_test_split(X, y, test_size=0.3, random_state=seed)

In [ ]:
# Imputação dos dados
#imputer = SimpleImputer(strategy='mean')  # ou 'median', 'most_frequent'

# Ajustar e transformar os dados de treino
#X_train_imputed = imputer.fit_transform(X_train)

# Transformar os dados de teste (apenas transformação, sem ajuste)
#X_test_imputed = imputer.transform(X_test)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score

# Algoritmos
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

results = []
names = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, shuffle=True, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results.append(accuracy)
    names.append(name)
    print(f"{name}: Accuracy {accuracy} / mean: {cv_results.mean()} / std: {cv_results.std()}")


LR: Accuracy 0.49532224532224534 / mean: 0.4762711780146357 / std: 0.016603035095017767
LDA: Accuracy 0.8118503118503119 / mean: 0.8157756323576202 / std: 0.017273823479776945
KNN: Accuracy 0.41424116424116425 / mean: 0.41456063076678334 / std: 0.019872145714147582
CART: Accuracy 1.0 / mean: 1.0 / std: 0.0
NB: Accuracy 0.49428274428274427 / mean: 0.47582624085268854 / std: 0.015488068366760865
SVM: Accuracy 0.5051975051975052 / mean: 0.4842884982500795 / std: 0.02063451302803724


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Árvore de Decisão (CART)
cart_model = DecisionTreeClassifier()
cart_model.fit(X_train, y_train)
cart_predictions = cart_model.predict(X_test)

print("Árvore de Decisão (CART):")
print("Precisão:", accuracy_score(y_test, cart_predictions))
print("Matriz de Confusão:\n", confusion_matrix(y_test, cart_predictions))
print("Relatório de Classificação:\n", classification_report(y_test, cart_predictions))

print(30*'--')

# Análise Discriminante Linear (LDA)
lda_model = LinearDiscriminantAnalysis()
lda_model.fit(X_train, y_train)
lda_predictions = lda_model.predict(X_test)

print("\nAnálise Discriminante Linear (LDA):")
print("Precisão:", accuracy_score(y_test, lda_predictions))
print("Matriz de Confusão:\n", confusion_matrix(y_test, lda_predictions))
print("Relatório de Classificação:\n", classification_report(y_test, lda_predictions))


Árvore de Decisão (CART):
Precisão: 1.0
Matriz de Confusão:
 [[335   0   0]
 [  0 953   0]
 [  0   0 636]]
Relatório de Classificação:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00       335
           2       1.00      1.00      1.00       953
           3       1.00      1.00      1.00       636

    accuracy                           1.00      1924
   macro avg       1.00      1.00      1.00      1924
weighted avg       1.00      1.00      1.00      1924

------------------------------------------------------------

Análise Discriminante Linear (LDA):
Precisão: 0.8118503118503119
Matriz de Confusão:
 [[ 57 211  67]
 [ 15 928  10]
 [ 37  22 577]]
Relatório de Classificação:
               precision    recall  f1-score   support

           1       0.52      0.17      0.26       335
           2       0.80      0.97      0.88       953
           3       0.88      0.91      0.89       636

    accuracy                           0.8

Os resultados indicam o desempenho dos modelos de Árvore de Decisão (CART) e Análise Discriminante Linear (LDA) nos dados de teste:

**Árvore de Decisão (CART):**

- Precisão: 1.0
- Matriz de Confusão: A matriz de confusão mostra que todas as previsões foram corretas. Não houve falsos positivos ou falsos negativos para nenhuma das classes.
- Relatório de Classificação: O relatório mostra que o modelo alcançou uma precisão, recall e pontuação F1 de 100% para todas as classes. Isso indica que o modelo teve um desempenho perfeito nos dados de teste.

**Análise Discriminante Linear (LDA):**

- Precisão: 0.81185
- Matriz de Confusão: A matriz de confusão mostra a distribuição das previsões erradas. Por exemplo, houve 211 falsos negativos (classe 1 erroneamente classificada como classe 2) e 67 falsos positivos (classe 1 erroneamente classificada como classe 3).
- Relatório de Classificação: O relatório mostra métricas de precisão, recall e pontuação F1 para cada classe. A precisão, recall e pontuação F1 para a classe 1 são mais baixas em comparação com as outras classes, o que indica que o modelo tem dificuldade em distinguir a classe 1 das outras classes.

---

Esses resultados indicam que a Árvore de Decisão teve um desempenho perfeito nos dados de teste, enquanto a Análise Discriminante Linear teve um desempenho inferior, com alguns erros de classificação, especialmente para a classe 1.

In [ ]:
import joblib

# Save the model as a pickle in a file
joblib.dump(cart_model, 'filename.pkl')

# Load the model from the file
cart_from_joblib = joblib.load('filename.pkl')

# Use the loaded model to make predictions
new_test = [
    [4.07, 4.05, 4.09, 4.03, 35740000.0, 0.48, 5],
    [25.40, 26.09, 26.67, 25.40, 307000000.0, -3.18, 2],
    [35.19, 35.51, 35.52, 34.32, 626000000.0, -0.34, 2],
    [37.36, 37.32, 37.43, 37.13, 1959000000.0, 0.08, 6],
    [17.16, 17.09, 17.22, 16.94, 4593000000.0, 0.54, 6],
]

cart_from_joblib.predict(new_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1, 2, 2, 1, 1])

In [ ]:
path_training = path + '/training.pkl'

cart_from_joblib = joblib.load(path_training)
cart_from_joblib.predict(new_test)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1])